In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("exemplo bla bla").config("spark.driver.memory","4g").config("spark.executor.memory","4g").getOrCreate()

23/03/20 14:37:57 WARN Utils: Your hostname, IC1019-1 resolves to a loopback address: 127.0.1.1; using 172.30.15.89 instead (on interface eth0)
23/03/20 14:37:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 14:37:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/20 14:38:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/20 14:38:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [ ]:
# Some general imports 

import os
import sys

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
# from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def plotHistogram(df, xcol, huecol=None):
    sns.histplot(data=df, x=xcol, hue=huecol, multiple="stack")

In [ ]:
def plot(df, xcol, ycol):
    sns.lineplot(data=df, x=xcol, y=ycol)

In [ ]:
def plotBar(df, xcol, ycol, huecol=None):
    sns.barplot(data=df, x=xcol, y=ycol, hue=huecol)

In [ ]:
def plotScatter(df, xcol, ycol, huecol=None):
    sns.scatterplot(data=df, x=xcol, y=ycol, hue=huecol)

In [ ]:
def plotScatterMatrix(df, huecol=None):
    sns.pairplot(data=df, hue=huecol)

In [ ]:
def plotCorrelationMatrix_1(df, annot=False):
    # compute the correlation matrix
    corr = df.corr()
    
    # generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    #cmap='coolwarm'

    # draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=annot,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
def plotCorrelationMatrix_2(df):
    # compute a correlation matrix and convert to long-form
    corr_mat = df.corr().stack().reset_index(name="correlation")
    # draw each cell as a scatter point with varying size and color
    g = sns.relplot(
        data=corr_mat,
        x="level_0", y="level_1", hue="correlation", size="correlation",
        palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
        height=10, sizes=(50, 250), size_norm=(-.2, .8),
    )

    # tweak the figure to finalize
    g.set(xlabel="", ylabel="", aspect="equal")
    g.despine(left=True, bottom=True)
    g.ax.margins(.02)
    for label in g.ax.get_xticklabels():
        label.set_rotation(90)
    for artist in g.legend.legendHandles:
        artist.set_edgecolor(".7")

In [ ]:
# Some Spark related imports we will use hereafter

import sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
filename = "train.csv"
df_train = spark.read.csv(filename,header=True, sep=",", inferSchema=True)


In [ ]:
filename = "test.csv"
df_teste = spark.read.csv(filename,header=True, sep=",", inferSchema=True)

In [ ]:
filename = "train_labels.csv"
df_train_labs = spark.read.csv(filename,header=True, sep=",", inferSchema=True)

In [ ]:
df_teste.printSchema()
df_teste.show(1, vertical=True)
teste_count = df_teste.count()
df_teste.show(1)

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- page: double (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- hover_duration: double (nullable = true)
 |-- text: string (nullable = true)
 |-- fqid: string (nullable = true)
 |-- room_fqid: string (nullable = true)
 |-- text_fqid: string (nullable = true)
 |-- fullscreen: string (nullable = true)
 |-- hq: string (nullable = true)
 |-- music: string (nullable = true)
 |-- level_group: string (nullable = true)
 |-- session_level: string (nullable = true)

-RECORD 0------------------------------
 session_id     | 20090109393214576    
 index          | 0                    
 elapsed_time   | 0   

In [ ]:
df_train.printSchema()
df_train.show(1, vertical=True)
train_count = df_train.count()
train_count
df_train.show(1)

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- page: double (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- hover_duration: double (nullable = true)
 |-- text: string (nullable = true)
 |-- fqid: string (nullable = true)
 |-- room_fqid: string (nullable = true)
 |-- text_fqid: string (nullable = true)
 |-- fullscreen: string (nullable = true)
 |-- hq: string (nullable = true)
 |-- music: string (nullable = true)
 |-- level_group: string (nullable = true)

-RECORD 0------------------------------
 session_id     | 20090312431273200    
 index          | 0                    
 elapsed_time   | 0                    
 event_name     | cutscene_

+-----------------+-----+------------+--------------+-----+-----+----+-------------------+-------------------+-------------+-------------+--------------+---------+-----+--------------------+--------------------+----------+----+-----+-----------+
|       session_id|index|elapsed_time|    event_name| name|level|page|        room_coor_x|        room_coor_y|screen_coor_x|screen_coor_y|hover_duration|     text| fqid|           room_fqid|           text_fqid|fullscreen|  hq|music|level_group|
+-----------------+-----+------------+--------------+-----+-----+----+-------------------+-------------------+-------------+-------------+--------------+---------+-----+--------------------+--------------------+----------+----+-----+-----------+
|20090312431273200|    0|           0|cutscene_click|basic|    0|null|-413.99140522263224|-159.31468617022784|        380.0|        494.0|          null|undefined|intro|tunic.historicals...|tunic.historicals...|      null|null| null|        0-4|
+---------------

In [ ]:
df_train_labs.show(5)

+--------------------+-------+
|          session_id|correct|
+--------------------+-------+
|20090312431273200_q1|      1|
|20090312433251036_q1|      0|
|20090314121766812_q1|      1|
|20090314363702160_q1|      1|
|20090314441803444_q1|      1|
+--------------------+-------+
only showing top 5 rows



In [ ]:
print('\nNulls in all columns')
cols_to_forget = ['Credit Limit']
teste_cols_interest = [x for x in df_teste.columns]
for cl in teste_cols_interest:
    k = df_teste.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {teste_count} records ({k*100/teste_count:.2f}%)')


Nulls in all columns
Column page with 3575 nulls or NaN, out of 3728 records (95.90%)
Column room_coor_x with 362 nulls or NaN, out of 3728 records (9.71%)
Column room_coor_y with 362 nulls or NaN, out of 3728 records (9.71%)
Column screen_coor_x with 362 nulls or NaN, out of 3728 records (9.71%)
Column screen_coor_y with 362 nulls or NaN, out of 3728 records (9.71%)
Column hover_duration with 3375 nulls or NaN, out of 3728 records (90.53%)
Column text with 2566 nulls or NaN, out of 3728 records (68.83%)
Column fqid with 1226 nulls or NaN, out of 3728 records (32.89%)
Column room_fqid with 6 nulls or NaN, out of 3728 records (0.16%)
Column text_fqid with 2572 nulls or NaN, out of 3728 records (68.99%)
Column fullscreen with 3728 nulls or NaN, out of 3728 records (100.00%)
Column hq with 3728 nulls or NaN, out of 3728 records (100.00%)
Column music with 3728 nulls or NaN, out of 3728 records (100.00%)
Column level_group with 6 nulls or NaN, out of 3728 records (0.16%)


In [ ]:
print('\nNulls in all columns')
cols_to_forget = ['Credit Limit']
train_cols_interest = [x for x in df_train.columns]
for cl in train_cols_interest:
    k = df_train.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {train_count} records ({k*100/train_count:.2f}%)')


Nulls in all columns


Column page with 12889465 nulls or NaN, out of 13174211 records (97.84%)


Column room_coor_x with 1036240 nulls or NaN, out of 13174211 records (7.87%)


Column room_coor_y with 1036240 nulls or NaN, out of 13174211 records (7.87%)


Column screen_coor_x with 1036240 nulls or NaN, out of 13174211 records (7.87%)


...

Column text_fqid with 8391056 nulls or NaN, out of 13174211 records (63.69%)


Column fullscreen with 13174211 nulls or NaN, out of 13174211 records (100.00%)


Column hq with 13174211 nulls or NaN, out of 13174211 records (100.00%)


Column music with 13174211 nulls or NaN, out of 13174211 records (100.00%)


Column level_group with 23790 nulls or NaN, out of 13174211 records (0.18%)


In [ ]:
df_train.head(4)

In [ ]:
df_train.tail(4)

[Row(session_id=22100221145014656, index=1601, elapsed_time=5485166, event_name='navigate_click', name='undefined', level=22, page=None, room_coor_x=332.69606975910096, room_coor_y=141.49317778482262, screen_coor_x=545.0, screen_coor_y=221.0, hover_duration=None, text=None, fqid='chap4_finale_c', room_fqid='tunic.capitol_2.hall', text_fqid=None, fullscreen=None, hq=None, music=None, level_group='13-22'),
 Row(session_id=22100221145014656, index=1602, elapsed_time=5485917, event_name='navigate_click', name='undefined', level=22, page=None, room_coor_x=369.9128589444621, room_coor_y=140.56920467904712, screen_coor_x=611.0, screen_coor_y=217.0, hover_duration=None, text=None, fqid=None, room_fqid='tunic.capitol_2.hall', text_fqid=None, fullscreen=None, hq=None, music=None, level_group='13-22'),
 Row(session_id=22100221145014656, index=1603, elapsed_time=5486753, event_name='navigate_click', name='undefined', level=22, page=None, room_coor_x=252.2996530220907, room_coor_y=123.8058891753311

In [ ]:
train_cols_interest.remove('fullscreen')
train_cols_interest.remove('music')
train_cols_interest.remove('hq')
train_cols_interest.remove('page')
train_cols_interest.remove('hover_duration')
train_cols_interest.remove('text')
train_cols_interest.remove('text_fqid')

In [ ]:
for cl in train_cols_interest:
    k = df_train.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {train_count} records ({k*100/train_count:.2f}%)')

Column room_coor_x with 1036240 nulls or NaN, out of 13174211 records (7.87%)


Column room_coor_y with 1036240 nulls or NaN, out of 13174211 records (7.87%)


Column screen_coor_x with 1036240 nulls or NaN, out of 13174211 records (7.87%)


Column screen_coor_y with 1036240 nulls or NaN, out of 13174211 records (7.87%)


Column fqid with 4171591 nulls or NaN, out of 13174211 records (31.66%)


Column room_fqid with 23790 nulls or NaN, out of 13174211 records (0.18%)


Column level_group with 23790 nulls or NaN, out of 13174211 records (0.18%)


In [ ]:
teste_cols_interest.remove('fullscreen')
teste_cols_interest.remove('music')
teste_cols_interest.remove('hq')
teste_cols_interest.remove('page')

In [ ]:
for cl in teste_cols_interest:
    k = df_teste.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {teste_count} records ({k*100/teste_count:.2f}%)')

Column room_coor_x with 362 nulls or NaN, out of 3728 records (9.71%)
Column room_coor_y with 362 nulls or NaN, out of 3728 records (9.71%)
Column screen_coor_x with 362 nulls or NaN, out of 3728 records (9.71%)
Column screen_coor_y with 362 nulls or NaN, out of 3728 records (9.71%)
Column hover_duration with 3375 nulls or NaN, out of 3728 records (90.53%)
Column text with 2566 nulls or NaN, out of 3728 records (68.83%)
Column fqid with 1226 nulls or NaN, out of 3728 records (32.89%)
Column room_fqid with 6 nulls or NaN, out of 3728 records (0.16%)
Column text_fqid with 2572 nulls or NaN, out of 3728 records (68.99%)
Column level_group with 6 nulls or NaN, out of 3728 records (0.16%)
Column session_level with 6 nulls or NaN, out of 3728 records (0.16%)


In [ ]:
print('\nUniqueness in teste:')
for cl in teste_cols_interest:
    k = df_teste.select(cl).distinct().count()
    print(f"Columns {cl} with {k} distinct values, out of {teste_count}"+f" records ({k*100/teste_count:.2f})%")


Uniqueness in teste:
Columns session_id with 3 distinct values, out of 3728 records (0.08)%
Columns index with 1564 distinct values, out of 3728 records (41.95)%
Columns elapsed_time with 3720 distinct values, out of 3728 records (99.79)%
Columns event_name with 11 distinct values, out of 3728 records (0.30)%
Columns name with 6 distinct values, out of 3728 records (0.16)%
Columns level with 23 distinct values, out of 3728 records (0.62)%
Columns room_coor_x with 3147 distinct values, out of 3728 records (84.42)%
Columns room_coor_y with 2669 distinct values, out of 3728 records (71.59)%
Columns screen_coor_x with 802 distinct values, out of 3728 records (21.51)%
Columns screen_coor_y with 578 distinct values, out of 3728 records (15.50)%
Columns hover_duration with 222 distinct values, out of 3728 records (5.95)%
Columns text with 499 distinct values, out of 3728 records (13.39)%
Columns fqid with 113 distinct values, out of 3728 records (3.03)%
Columns room_fqid with 20 distinct val

In [ ]:
print('\nUniqueness in treino:')
for cl in train_cols_interest:
    k = df_train.select(cl).distinct().count()
    print(f"Columns {cl} with {k} distinct values, out of {train_count}"+f" records ({k*100/train_count:.2f})%")


Uniqueness in treino:


Columns session_id with 11779 distinct values, out of 13174211 records (0.09)%


Columns index with 20348 distinct values, out of 13174211 records (0.15)%


Columns elapsed_time with 3804019 distinct values, out of 13174211 records (28.87)%


Columns event_name with 11 distinct values, out of 13174211 records (0.00)%


...

Columns screen_coor_x with 35332 distinct values, out of 13174211 records (0.27)%


Columns screen_coor_y with 56859 distinct values, out of 13174211 records (0.43)%


Columns fqid with 128 distinct values, out of 13174211 records (0.00)%


Columns room_fqid with 20 distinct values, out of 13174211 records (0.00)%


Columns level_group with 4 distinct values, out of 13174211 records (0.00)%


In [ ]:
df_train_on = df_train.select(train_cols_interest)
df_teste_on = df_teste.select(teste_cols_interest)

In [ ]:
df_train_on.printSchema()
df_teste_on.printSchema()

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- fqid: string (nullable = true)
 |-- room_fqid: string (nullable = true)
 |-- level_group: string (nullable = true)

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- hover_duration: double (nullable =